# GPTQ 量化算法（广义剪枝训练量化）

你可以在这个 [笔记本](https://colab.research.google.com/drive/1_TIrmuKOFhuRRiTWN94iLKUFu6ZX4ceb?usp=sharing) 中尝试使用 PEFT 进行 GPTQ 量化，并在 [这篇博客文章](https://huggingface.co/blog/gptq-integration) 中了解更多细节！

[AutoGPTQ](https://github.com/PanQiWei/AutoGPTQ) 库实现了 GPTQ 算法，这是一种后训练量化技术，其中权重矩阵的每一行独立量化，以找到一个能够最小化误差的权重版本。这些权重被量化为 int4，但在推理时会动态恢复为 fp16。这可以将内存使用量减少 4 倍，因为 int4 权重是在融合内核中而不是在 GPU 的全局内存中反量化，还可以期待推理速度的提升，因为使用较低的比特宽度通信时间更短。

在开始之前，请确保安装了以下库：


In [ ]:
pip install auto-gptq
pip install --upgrade accelerate optimum transformers


要量化模型（目前仅支持文本模型），需要创建一个 [GPTQConfig](/docs/transformers/v4.46.3/en/main_classes/quantization#transformers.GPTQConfig) 类，并设置量化位数、用于校准权重的 数据集和分词器。


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

model_id = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_id)
gptq_config = GPTQConfig(bits=4, dataset="c4", tokenizer=tokenizer)


你也可以传递自己的数据集作为字符串列表，但强烈建议使用 GPTQ 论文中相同的数据集。


In [ ]:
dataset = ["auto-gptq 是一个易于使用的模型量化库，具有用户友好的 API，基于 GPTQ 算法。"]
gptq_config = GPTQConfig(bits=4, dataset=dataset, tokenizer=tokenizer)


加载要量化的模型，并将 `gptq_config` 传递给 [from_pretrained()](/docs/transformers/v4.46.3/en/model_doc/auto#transformers.AutoModel.from_pretrained) 方法。设置 `device_map="auto"` 可以自动将模型卸载到 CPU 以帮助模型适应内存，并允许模型模块在 CPU 和 GPU 之间移动以进行量化。


In [ ]:
quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=gptq_config)


如果你因为数据集太大而内存不足，磁盘卸载不被支持。如果出现这种情况，可以通过传递 `max_memory` 参数来分配设备（GPU 和 CPU）上的内存使用量。


In [ ]:
quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto", 
    max_memory={0: "30GiB", 1: "46GiB", "cpu": "30GiB"}, 
    quantization_config=gptq_config
)


根据你的硬件，从头开始量化模型可能需要一些时间。在免费的 Google Colab GPU 上量化 [facebook/opt-350m](https://huggingface.co/facebook/opt-350m) 模型大约需要 5 分钟，但在 NVIDIA A100 上量化一个 175B 参数的模型则需要约 4 小时。在量化模型之前，最好检查一下 Hugging Face Hub 是否已经存在 GPTQ 量化的版本。

一旦模型被量化，你可以将模型和分词器推送到 Hub 以便轻松共享和访问。使用 [push_to_hub()](/docs/transformers/v4.46.3/en/main_classes/model#transformers.utils.PushToHubMixin.push_to_hub) 方法保存 [GPTQConfig](/docs/transformers/v4.46.3/en/main_classes/quantization#transformers.GPTQConfig)：


In [ ]:
quantized_model.push_to_hub("opt-125m-gptq")
tokenizer.push_to_hub("opt-125m-gptq")


你也可以使用 [save_pretrained()](/docs/transformers/v4.46.3/en/main_classes/model#transformers.PreTrainedModel.save_pretrained) 方法将量化后的模型本地保存。如果模型是在设置了 `device_map` 参数的情况下量化的，在保存之前请确保将整个模型移动到 GPU 或 CPU。例如，要在 CPU 上保存模型：


In [ ]:
quantized_model.save_pretrained("opt-125m-gptq")
tokenizer.save_pretrained("opt-125m-gptq")

# 如果量化时设置了 device_map
quantized_model.to("cpu")
quantized_model.save_pretrained("opt-125m-gptq")


使用 [from_pretrained()](/docs/transformers/v4.46.3/en/main_classes/model#transformers.PreTrainedModel.from_pretrained) 方法重新加载量化后的模型，并设置 `device_map="auto"` 以自动将模型分布在所有可用的 GPU 上，从而更快地加载模型且不占用过多内存。


In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("{your_username}/opt-125m-gptq", device_map="auto")


## ExLlama

[ExLlama](https://github.com/turboderp/exllama) 是 Llama 模型的 Python/C++/CUDA 实现，旨在使用 4 位 GPTQ 权重进行更快的推理（参见这些 [基准测试](https://github.com/huggingface/optimum/tree/main/tests/benchmark#gptq-benchmark)）。当创建 [GPTQConfig](/docs/transformers/v4.46.3/en/main_classes/quantization#transformers.GPTQConfig) 对象时，默认激活 ExLlama 内核。为了进一步提高推理速度，可以使用 [ExLlamaV2](https://github.com/turboderp/exllamav2) 内核，通过配置 `exllama_config` 参数实现：


In [ ]:
import torch
from transformers import AutoModelForCausalLM, GPTQConfig

gptq_config = GPTQConfig(bits=4, exllama_config={"version":2})
model = AutoModelForCausalLM.from_pretrained("{your_username}/opt-125m-gptq", device_map="auto", quantization_config=gptq_config)


仅支持 4 位模型，如果你正在使用 PEFT 微调量化模型，建议禁用 ExLlama 内核。

只有当整个模型都在 GPU 上时，ExLlama 内核才受支持。如果你在 CPU 上使用 AutoGPTQ（版本 > 0.4.2）进行推理，则需要禁用 ExLlama 内核。这将覆盖 `config.json` 文件中的量化配置中与 ExLlama 内核相关的属性。


In [ ]:
import torch
from transformers import AutoModelForCausalLM, GPTQConfig

gptq_config = GPTQConfig(bits=4, use_exllama=False)
model = AutoModelForCausalLM.from_pretrained("{your_username}/opt-125m-gptq", device_map="cpu", quantization_config=gptq_config)